### Notebook for concept detection in neural network

In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
import numpy as np
import os
import sys
from tqdm import tqdm
import json
import random
import string

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

import concepts
import env
from policy import ActorCriticNet

model_name = "net"
session_name = "starship"
board_size = 5
board_name = f'{board_size}x{board_size}'

agents_to_sample = [0, 150, 300, 450, 600, 750, 900, 1050]

full_model_path = f"../models/saved_sessions/board_size_{board_size}/{session_name}/"

CONCEPT_FUNC = concepts.concept_two_eyes

CONCEPT_NAME = concepts.concept_two_eyes.__name__


In [40]:
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [44]:
def play_match(agents: list[ActorCriticNet], board_size, concept_function):
    go_env = env.GoEnv(board_size)
    state = go_env.reset()

    player_to_start = 1 if np.random.random() > 0.5 else 0
    current_player = player_to_start

    total_moves = board_size * board_size * 4
    moves = 0

    positive_cases = []
    negative_cases = []

    sample_ratio = 0.2

    game_over = False

    while not game_over:
        if moves == total_moves:
            break
        # Remove array index 3 and 5 from the current state making it an shape of (4, 5, 5)
        state_copy = np.delete(state, [3, 5], axis=0)
        
        if np.random.random() < sample_ratio:
            if concept_function(state):
                positive_cases.append(state_copy)
            else:
                negative_cases.append(state_copy)
        
        action = agents[current_player].best_action(state)

        state, _, game_over, _ = go_env.step(action)
        moves += 1

        current_player = (current_player + 1) % 2
    
    return positive_cases, negative_cases

positive_cases = []
negative_cases = []

CASES_TO_SAMPLE = 2500 # 25000

positive_bar = tqdm(total=CASES_TO_SAMPLE, desc="Positive cases")

while len(positive_cases) < CASES_TO_SAMPLE or len(negative_cases) < CASES_TO_SAMPLE:
    for i in range(len(agents)):
        for j in range(i + 1, len(agents)):
            pos, neg = play_match([agents[i], agents[j]], board_size, CONCEPT_FUNC)
            positive_cases.extend(pos)
            negative_cases.extend(neg)
            positive_bar.update(len(pos))

positive_cases = positive_cases[:CASES_TO_SAMPLE]
negative_cases = negative_cases[:CASES_TO_SAMPLE]

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

Positive cases:   0%|          | 5/2500 [00:06<56:14,  1.35s/it]
Positive cases: 2501it [30:35,  1.26s/it]                           

In [45]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (2500, 4, 5, 5)
Negative cases:  (2500, 4, 5, 5)


In [46]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [47]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  2437
Duplicate count in negative:  1349


In [ ]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print()

In [48]:
# Positions to consider are 80% of the total positions
POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0] * 2)
print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
#POSITIONS_TO_CONSIDER = 4000 #40000
VALIDATION_POSITIONS = 10000 #10000

BINARY = True

Positions to consider: 4000


In [49]:
# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

# Use the regression
score = concepts.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=BINARY
)

print("Regression score: ", score)

# Remove the files if they exist
if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
    # Test if epoch folder exists
    if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
        # Remove all files in the epoch folder
        # Fist save the 
        for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
            os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name, file))

os.makedirs("../concept_presences", exist_ok=True)
os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name), exist_ok=True)

random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

# Save concept presences in json file
with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, name, random_suffix), "w") as f:
    json.dump(score, f)

Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.8737 - val_loss: 0.8426
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8349 - val_loss: 0.8122
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8050 - val_loss: 0.7897
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7829 - val_loss: 0.7737
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7662 - val_loss: 0.7613
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7529 - val_loss: 0.7519
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7424 - val_loss: 0.7441
Epoch 8/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7336 - val_loss: 0.7372
Epoch 9/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7257 - val_loss: 0.7310
Epoch 10/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7187 - val_loss: 0.7254

In [50]:
epochs_to_look_at = [0, 150, 300, 450, 600, 750, 900, 1050]

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_resblock_outputs(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = concepts.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
        # Test if epoch folder exists
        if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
            # Remove all files in the epoch folder
            # Fist save the 
            for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
                os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch, file))

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, epoch, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)




Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 270.42it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 1s 2ms/step - loss: 0.9777 - val_loss: 0.9510
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.9247 - val_loss: 0.9015
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8777 - val_loss: 0.8573
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8359 - val_loss: 0.8183
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7990 - val_loss: 0.7840
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7671 - val_loss: 0.7550
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7410 - val_loss: 0.7319
Epoch 8/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7202 - val_loss: 0.7136
Epoch 9/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7044 - val_loss: 0.7008
Epoch 10/50
125/125 [==============================] - 0s 1ms/step

Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 279.84it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.9482 - val_loss: 0.9059
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8843 - val_loss: 0.8624
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8464 - val_loss: 0.8284
Epoch 4/50
125/125 [==============================] - 0s 936us/step - loss: 0.8147 - val_loss: 0.7986
Epoch 5/50
125/125 [==============================] - 0s 921us/step - loss: 0.7868 - val_loss: 0.7709
Epoch 6/50
125/125 [==============================] - 0s 953us/step - loss: 0.7612 - val_loss: 0.7468
Epoch 7/50
125/125 [==============================] - 0s 913us/step - loss: 0.7389 - val_loss: 0.7254
Epoch 8/50
125/125 [==============================] - 0s 932us/step - loss: 0.7190 - val_loss: 0.7058
Epoch 9/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7004 - val_loss: 0.6882
Epoch 10/50
125/125 [==============================] - 0

Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 287.00it/s]


...

Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.9699 - val_loss: 0.8991
Epoch 2/50
125/125 [==============================] - 0s 997us/step - loss: 0.8756 - val_loss: 0.8478
Epoch 3/50
125/125 [==============================] - 0s 994us/step - loss: 0.8294 - val_loss: 0.8096
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7918 - val_loss: 0.7765
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7593 - val_loss: 0.7483
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7314 - val_loss: 0.7236
Epoch 7/50
125/125 [==============================] - 0s 964us/step - loss: 0.7071 - val_loss: 0.7021
Epoch 8/50
125/125 [==============================] - 0s 992us/step - loss: 0.6859 - val_loss: 0.6829
Epoch 9/50
125/125 [==============================] - 0s 966us/step - loss: 0.6672 - val_loss: 0.6665
Epoch 10/50
125/125 [==============================] - 0

Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 289.07it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 1.0371 - val_loss: 0.9809
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.9338 - val_loss: 0.9122
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8728 - val_loss: 0.8582
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8244 - val_loss: 0.8140
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7842 - val_loss: 0.7767
Epoch 6/50
125/125 [==============================] - 0s 988us/step - loss: 0.7497 - val_loss: 0.7438
Epoch 7/50
125/125 [==============================] - 0s 999us/step - loss: 0.7196 - val_loss: 0.7147
Epoch 8/50
125/125 [==============================] - 0s 948us/step - loss: 0.6935 - val_loss: 0.6904
Epoch 9/50
125/125 [==============================] - 0s 973us/step - loss: 0.6711 - val_loss: 0.6687
Epoch 10/50
125/125 [==============================] - 0s 

Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 288.72it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.9793 - val_loss: 0.9216
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8996 - val_loss: 0.8612
Epoch 3/50
125/125 [==============================] - 0s 999us/step - loss: 0.8472 - val_loss: 0.8152
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8045 - val_loss: 0.7759
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7684 - val_loss: 0.7438
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7370 - val_loss: 0.7146
Epoch 7/50
125/125 [==============================] - 0s 978us/step - loss: 0.7092 - val_loss: 0.6890
Epoch 8/50
125/125 [==============================] - 0s 980us/step - loss: 0.6844 - val_loss: 0.6663
Epoch 9/50
125/125 [==============================] - 0s 965us/step - loss: 0.6632 - val_loss: 0.6468
Epoch 10/50
125/125 [==============================] - 0s 